# IR_best – Single Notebook (GitHub Ready)
_Last updated: 2025-08-26 02:22:34_

**Structure overview (each section = one task):**
1. **Environment Setup** – optional `requirements.txt` 설치
2. **Project Paths & Seed** – 경로/시드 설정
3. **Data Loading** – 데이터 불러오기
4. **Training / Fitting** – 모델 학습
5. **Evaluation / Inference** – 평가/추론
6. **Visualization** – 결과 시각화
7. **Saving Artifacts** – 모델/결과 저장


# 1) Environment Setup
_역할: `requirements.txt`가 있으면 설치. 실패해도 계속._

In [ ]:
# --- Environment Setup ---
"""
- 로컬/서버/코랩 어디서든 동작하도록 'requirements.txt'가 있으면 설치 시도(선택).
- 실패해도 노트북 실행은 계속됩니다.
"""
import sys, subprocess
from pathlib import Path

req = Path('requirements.txt')
if req.exists():
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', str(req)])
    except Exception as e:
        print('Warning: dependency install failed:', e)


# 2) Project Paths & Seed
_역할: `PROJECT_ROOT`, `DATA_DIR`, `OUTPUT_DIR` 설정 및 `set_seed()`._

In [ ]:
# --- Paths & Reproducibility ---
from pathlib import Path
import os, random

# 프로젝트 루트(레포 루트 가정). 필요 시 환경변수 PROJECT_ROOT 로 오버라이드.
PROJECT_ROOT = Path(os.getenv('PROJECT_ROOT', '.')).resolve()
DATA_DIR = PROJECT_ROOT / 'data'
OUTPUT_DIR = PROJECT_ROOT / 'outputs'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def p(*parts: str) -> Path:
    """Convenience: PROJECT_ROOT / parts"""
    return PROJECT_ROOT.joinpath(*parts)

def set_seed(seed: int = 42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    try:
        import numpy as np
        np.random.seed(seed)
    except Exception:
        pass
    try:
        import torch
        import torch.backends.cudnn as cudnn
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        cudnn.deterministic = True
        cudnn.benchmark = False
    except Exception:
        pass

set_seed(42)
print('PROJECT_ROOT:', PROJECT_ROOT)
print('DATA_DIR:', DATA_DIR)
print('OUTPUT_DIR:', OUTPUT_DIR)


# 3) Data Loading
_역할: 원천 데이터 로딩/전처리. 경로는 `p(...)` 사용._

In [ ]:
# --- Data Loading ---
import os, json
from collections import defaultdict

root = "./Set-A"   # 데이터셋 루트
img_train = os.path.join(root, "images/train")
img_val   = os.path.join(root, "images/val")
lab_train = os.path.join(root, "labels/train")
lab_val   = os.path.join(root, "labels/val")
os.makedirs(lab_train, exist_ok=True)
os.makedirs(lab_val, exist_ok=True)

# labels 폴더 안에서 json 탐색
json_files = [os.path.join(root, "labels", f) for f in os.listdir(os.path.join(root,"labels")) if f.endswith(".json")]
print("발견된 JSON:", json_files)

def convert_coco(json_path):
    with open(json_path,"r",encoding="utf-8") as f:
        coco = json.load(f)

    # person category id
    person_ids = [c["id"] for c in coco["categories"] if c["name"].lower()=="person"]

    imginfo = {im["id"]: (im["file_name"], im["width"], im["height"]) for im in coco["images"]}

    # annotation 변환
    outputs = defaultdict(list)
    for ann in coco["annotations"]:
        if ann["category_id"] not in person_ids:
            continue
        img_id = ann["image_id"]
        fn, w, h = imginfo[img_id]
        x,y,bw,bh = ann["bbox"]

        x_c = (x+bw/2)/w
        y_c = (y+bh/2)/h
        ww  = bw/w
        hh  = bh/h
        line = f"0 {x_c:.6f} {y_c:.6f} {ww:.6f} {hh:.6f}\n"

        stem, _ = os.path.splitext(os.path.basename(fn))
        outputs[stem].append(line)

    # 저장
    for stem, lines in outputs.items():
        # train/val 어느 쪽에 속하는지 체크
        if os.path.exists(os.path.join(img_train, stem+".jpg")):
            outpath = os.path.join(lab_train, stem+".txt")
        else:
            outpath = os.path.join(lab_val, stem+".txt")
        with open(outpath,"w") as f:
            f.writelines(lines)

for jp in json_files:
    convert_coco(jp)

print("변환 완료. 샘플 몇 개 확인:")
import glob
for path in glob.glob(os.path.join(lab_train,"*.txt"))[:5]:
    print(path, "=>", open(path).read().strip())


In [ ]:
# --- Data Loading ---
import cv2, os, glob
from tqdm import tqdm

def resize_dir(src_dir, dst_dir, size=(128,128)):
    os.makedirs(dst_dir, exist_ok=True)
    count = 0
    for ext in ("*.jpg","*.jpeg","*.png","*.bmp"):
        for path in glob.glob(os.path.join(src_dir, ext)):
            img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
            if img is None:
                continue
            resized = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
            out_path = os.path.join(dst_dir, os.path.basename(path))
            cv2.imwrite(out_path, resized)
            count += 1
    return count

print("== 128x128 이미지 생성 ==")
pairs = [
    (os.path.join(root, "images/train"), os.path.join(root, "images_128/train")),
    (os.path.join(root, "images/val"),   os.path.join(root, "images_128/val")),
]
total = 0
for src, dst in pairs:
    n = resize_dir(src, dst, (128,128))
    print(f"{src} -> {dst} : {n} files")
    total += n
print("총 변환 수:", total)


In [ ]:
# --- Data Loading ---
root = "./Set-A"
yaml_text = f"""# IR person detection (128x128) dataset
path: {root}
train: images/train
val: images/val

nc: 1
names: [person]
"""
yaml_path = "./yolov5/data/ir128.yaml"
with open(yaml_path, "w") as f:
    f.write(yaml_text)
print("wrote:", yaml_path, "\n---\n", yaml_text)


In [ ]:
# --- Data Loading ---
import glob, os

labs = "./Set-A/labels"
empty = [p for p in glob.glob(os.path.join(labs, "**/*.txt"), recursive=True) if os.stat(p).st_size == 0]
print("빈 라벨 파일 수:", len(empty))
print("샘플:", empty[:5])


In [ ]:
# --- Data Loading ---
import os, re, time, subprocess, sys, shlex

# ===== 설정 =====
os.environ["WANDB_DISABLED"] = "true"   # W&B 끄기
os.environ["PYTHONUNBUFFERED"] = "1"    # 실시간 플러시

IMG, BATCH, EPOCHS = 128, 64, 150
YAML   = "./yolov5/data/ir128.yaml"
WEIGHT = "yolov5s.pt"
NAME   = "ir128_person_stream"

cmd = [
    "python", "./yolov5/train.py",
    "--img", str(IMG),
    "--batch", str(BATCH),
    "--epochs", str(EPOCHS),
    "--data", YAML,
    "--weights", WEIGHT,
    "--name", NAME,
    "--workers", "2",
    "--cache", "ram",
    "--rect"
]

print("▶", " ".join(shlex.quote(x) for x in cmd))
print("="*80)

# ===== 파서 준비 =====
total_epochs = None
cur_epoch = 0
t0 = time.time()
t_epoch = time.time()
expect_metrics_next = False

# YOLOv5 로그 패턴
re_total = re.compile(r"Starting training for (\d+) epochs")
re_metrics_header = re.compile(r"\s*Class\s+Images\s+Instances\s+P\s+R\s+mAP50\s+mAP50-95")
re_metrics_all    = re.compile(r"\s*all\s+\d+\s+\d+\s+([0-9.]+)\s+([0-9.]+)\s+([0-9.]+)\s+([0-9.]+)")

# ===== 실행 & 스트리밍 =====
proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

try:
    for line in proc.stdout:
        line = line.rstrip("\n")

        # 총 에폭 수 추출
        if total_epochs is None:
            m = re_total.search(line)
            if m:
                total_epochs = int(m.group(1))
                print(f"총 {total_epochs} 에폭 학습 시작… (실시간 요약 표시)")

        # 검증 헤더 다음 줄에서 metrics가 옴
        if re_metrics_header.search(line):
            expect_metrics_next = True
            continue

        if expect_metrics_next:
            expect_metrics_next = False
            m = re_metrics_all.search(line)
            if m:
                cur_epoch += 1
                P, R, m50, m95 = map(float, m.groups())
                now = time.time()
                epoch_time = now - t_epoch
                t_epoch = now
                elapsed = now - t0
                # ⬇️ 최소 요약 한 줄
                print(f"[{cur_epoch}/{total_epochs or '?'}] "
                      f"P={P:.3f} R={R:.3f} mAP50={m50:.3f} mAP50-95={m95:.3f} "
                      f"epoch_time={epoch_time:.1f}s elapsed={elapsed:.1f}s")
            continue

        # 중요 경고만 통과 (원하면 더 줄여도 됨)
        if "corrupt image/label" in line or "WARNING" in line and "wandb" not in line:
            print("WARN:", line)

        # 마지막 저장 경로 표시
        if "Results saved to" in line:
            print(line)

    proc.wait()
finally:
    if proc.poll() is None:
        proc.terminate()

print("="*80)
print("학습 종료 (returncode =", proc.returncode, ")")


In [ ]:
# --- Data Loading ---
# === GT vs Pred, 상위 10장 비교 시각화 ===
import os, glob, shutil, subprocess, cv2
import numpy as np
import matplotlib.pyplot as plt

# 1) 경로/가중치 결정
root = "./Set-A"
img_val = f"{root}/images_128/val" if os.path.isdir(f"{root}/images_128/val") else f"{root}/images/val"
lab_val = f"{root}/labels_128/val" if os.path.isdir(f"{root}/labels_128/val") else f"{root}/labels/val"

wdir = "./yolov5/runs/train/ir128_person_stream2/weights"
best, last = f"{wdir}/best.pt", f"{wdir}/last.pt"
weights = best if os.path.exists(best) else (last if os.path.exists(last) else "yolov5s.pt")
print("weights:", weights)
print("val img dir:", img_val)
print("val lab dir:", lab_val)

# 2) non-empty 라벨 10개 고르기
label_files = [p for p in glob.glob(os.path.join(lab_val, "*.txt")) if os.stat(p).st_size > 0]
label_files = sorted(label_files)[:10]
assert label_files, "val에 non-empty 라벨이 없습니다."

# 3) 선택한 10장만 임시 폴더로 복사 → detect 실행
tmp_src = "./_tmp_val10"
shutil.rmtree(tmp_src, ignore_errors=True); os.makedirs(tmp_src, exist_ok=True)
picked = []
for txt in label_files:
    stem = os.path.splitext(os.path.basename(txt))[0]
    for ext in (".jpg",".jpeg",".png",".bmp",".JPG",".PNG"):
        ip = os.path.join(img_val, stem+ext)
        if os.path.exists(ip):
            shutil.copy2(ip, os.path.join(tmp_src, os.path.basename(ip)))
            picked.append(os.path.basename(ip))
            break
print("picked images:", picked)

# 4) 모델 예측(detect.py) — conf 낮춰 확인
out_proj = "./yolov5/runs/vis"
out_name = "cmp10"
subprocess.run([
    "python", "./yolov5/detect.py",
    "--weights", weights,
    "--img", "128",
    "--conf", "0.01",      # 낮춰서 미검출 방지
    "--iou", "0.45",
    "--classes", "0",      # person만
    "--source", tmp_src,
    "--project", out_proj,
    "--name", out_name,
    "--exist-ok",
    "--line-thickness", "1",   # ← 박스 두께 얇게
    "--font-size", "10",       # ← 라벨 폰트 작게 (detect.py에 font_size 지원 추가해둔 전제)
    # "--hide-conf",           # ← (선택) 확신도 숫자 숨기기
], check=True)

# 5) GT 그리기 함수
def draw_gt(img_bgr, txt_path):
    H, W = img_bgr.shape[:2]
    if os.path.exists(txt_path) and os.stat(txt_path).st_size > 0:
        with open(txt_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                _, x, y, w, h = map(float, parts)
                cx, cy, ww, hh = x*W, y*H, w*W, h*H
                x1, y1 = int(cx-ww/2), int(cy-hh/2)
                x2, y2 = int(cx+ww/2), int(cy+hh/2)
                cv2.rectangle(img_bgr, (x1,y1), (x2,y2), (0,255,0), 2)  # GT: 초록
                cv2.putText(img_bgr, "GT", (x1, max(0,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)
    return img_bgr

# 6) 좌=GT, 우=Pred로 붙여 보여주기
pred_dir = os.path.join(out_proj, out_name)
plt.figure(figsize=(12, 20))
for i, fn in enumerate(picked):
    stem = os.path.splitext(fn)[0]

    ip = os.path.join(tmp_src, fn)
    gt_txt = os.path.join(lab_val, stem + ".txt")
    pr = os.path.join(pred_dir, fn)  # detect.py가 같은 파일명으로 저장

    # GT 이미지
    gt = cv2.imread(ip, cv2.IMREAD_UNCHANGED)
    if gt is None:
        continue
    if gt.ndim == 2:
        gt = cv2.cvtColor(gt, cv2.COLOR_GRAY2BGR)
    gt_drawn = draw_gt(gt.copy(), gt_txt)
    gt_rgb = cv2.cvtColor(gt_drawn, cv2.COLOR_BGR2RGB)

    # Pred 이미지
    pr_img = cv2.imread(pr)
    if pr_img is None:
        pr_img = np.zeros_like(gt)
    pr_rgb = cv2.cvtColor(pr_img, cv2.COLOR_BGR2RGB)

    # concat & plot
    plt.subplot(5, 2, i+1)
    combo = np.concatenate([gt_rgb, pr_rgb], axis=1)
    plt.imshow(combo)
    plt.title(f"{fn}   [GT | Pred]")
    plt.axis("off")

plt.tight_layout(); plt.show()


In [ ]:
# --- Data Loading ---
import zipfile
import os
from google.colab import files

zip_path = "./colab_backup.zip"

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    # 1) 폴더 통째로 추가
    for folder in ["yolov5", "Set-A"]:
        for root, _, files_ in os.walk(os.path.join(".", folder)):
            for file in files_:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, ".")  # 상대경로로 저장
                zf.write(file_path, arcname)

    # 2) 개별 파일 추가
    for f in ["your_thermal.jpg", "3.png", "4.png"]:
        file_path = os.path.join(".", f)
        if os.path.exists(file_path):
            zf.write(file_path, f)

# 3) 다운로드
files.download(zip_path)


# 4) Training / Fitting
_역할: 학습 루프/옵티마이저/스케줄러 등 모델 학습 수행._

In [ ]:
# --- Model Training / Fitting ---
# 런타임 체크 (Colab GPU 권장)
!nvidia-smi -L || echo "No GPU visible (OK for CPU training, but slower)"


In [ ]:
# --- Model Training / Fitting ---
import os
for p in ["./Set-A/images/train.cache",
          "./Set-A/images/val.cache"]:
    if os.path.exists(p):
        os.remove(p); print("삭제:", p)


In [ ]:
# --- Model Training / Fitting ---
import os, cv2, numpy as np, subprocess, matplotlib.pyplot as plt

img_in = "./your_thermal.jpg"
weights = "./yolov5/runs/train/ir128_person_stream2/weights/best.pt"

# 1) 그레이스케일 + CLAHE(대비증강) + 3채널
img = cv2.imread(img_in, cv2.IMREAD_COLOR); assert img is not None
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(gray)
proc = np.dstack([clahe, clahe, clahe])
prep_path = "./_prep_input_clahe.jpg"; cv2.imwrite(prep_path, proc)

# 2) 고해상도/낮은 conf로 3가지 조합 테스트
combos = [
    ("try128",  "128", "0.02"),
    ("try320",  "320", "0.02"),
    ("try640",  "640", "0.02"),  # 가장 관대하게
]
out_proj = "./yolov5/runs/vis"
paths = []
for name, imgz, conf in combos:
    subprocess.run([
        "python","./yolov5/detect.py",
        "--weights", weights if os.path.exists(weights) else "yolov5s.pt",
        "--img", imgz, "--conf", conf, "--iou", "0.45",
        "--classes","0", "--source", prep_path,
        "--project", out_proj, "--name", name, "--exist-ok",
        "--line-thickness","1"
    ], check=True)
    paths.append(os.path.join(out_proj, name, os.path.basename(prep_path)))

# 3) 결과 비교 표시
plt.figure(figsize=(12,4))
for i,p in enumerate(paths,1):
    im = cv2.imread(p);
    if im is None: im = proc.copy()
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.subplot(1,3,i); plt.imshow(im); plt.axis("off"); plt.title(combos[i-1][0])
plt.tight_layout(); plt.show()


In [ ]:
# --- Model Training / Fitting ---
import os, cv2, numpy as np, subprocess, matplotlib.pyplot as plt

img_in = "./4.png"
weights = "./yolov5/runs/train/ir128_person_stream2/weights/best.pt"

# 1) 그레이스케일 + CLAHE(대비증강) + 3채널
img = cv2.imread(img_in, cv2.IMREAD_COLOR); assert img is not None
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)).apply(gray)
proc = np.dstack([clahe, clahe, clahe])
prep_path = "./_prep_input_clahe.png"; cv2.imwrite(prep_path, proc)

# 2) try128만 실행
name, imgz, conf = ("try128", "128", "0.02")
out_proj = "./yolov5/runs/vis"
subprocess.run([
    "python","./yolov5/detect.py",
    "--weights", weights if os.path.exists(weights) else "yolov5s.pt",
    "--img", imgz, "--conf", conf, "--iou", "0.45",
    "--classes","0", "--source", prep_path,
    "--project", out_proj, "--name", name, "--exist-ok",
    "--line-thickness","1"
], check=True)

# 3) 결과 표시 (제목 없음)
out_path = os.path.join(out_proj, name, os.path.basename(prep_path))
im = cv2.imread(out_path)
if im is None: im = proc.copy()
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

plt.imshow(im)
plt.axis("off")
plt.show()


In [ ]:
# --- Model Training / Fitting ---
# 검증
!python val.py --weights runs/train/ir128_person_stream2/weights/best.pt --data ./yolov5/data/ir128.yaml --img 128


In [ ]:
# --- Model Training / Fitting ---
# 내보내기 (ONNX, TorchScript 등)
!python export.py --weights runs/train/ir128_person_stream2/weights/best.pt --include onnx torchscript --img 128 --simplify


# Appendix) Misc / Utilities
_역할: 유틸리티/보조 함수._

In [ ]:
# --- Misc / Utilities ---
# YOLOv5 설치 및 의존성
!git clone -q https://github.com/ultralytics/yolov5.git


In [ ]:
# --- Misc / Utilities ---
import os, zipfile

# 1) zip 파일 경로 지정 (업로드한 파일명에 맞게 수정)
zip_path = "./Set-A.zip"
root = "./SET-A"

# 2) 압축 해제
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall("./")   # 압축 풀면 ./SET-A 가 생성된다고 가정


In [ ]:
# --- Misc / Utilities ---
import os, shutil

root = "./Set-A"
img128 = os.path.join(root, "images_128")
lab128 = os.path.join(root, "labels_128")
imgs   = os.path.join(root, "images")
labs   = os.path.join(root, "labels")

# images_128 -> images
if os.path.exists(img128):
    if os.path.exists(imgs):
        shutil.rmtree(imgs)
    shutil.move(img128, imgs)
    print("moved:", img128, "->", imgs)

# labels_128 -> labels
if os.path.exists(lab128):
    if os.path.exists(labs):
        shutil.rmtree(labs)
    shutil.move(lab128, labs)
    print("moved:", lab128, "->", labs)


In [ ]:
# --- Misc / Utilities ---
import os
os.environ["WANDB_DISABLED"] = "true"
